In [201]:

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
import logging
import azureml.dataprep as dprep 
import azureml.core.dataset as Dataset
from azureml.core import Dataset

In [1]:
import pandas as pd 
print(pd.__version__)

0.23.4


In [2]:
pip install pandas==0.24.2


The following command must be run outside of the IPython shell:

    $ pip install pandas==0.24.2

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [202]:
data_df = pd.read_csv('data/Training_AI_ML_OrderExport_0701_0930_v2.csv')
test_df= pd.read_csv('data/Test_AI_ML_OrderExport_1001_1014_v2.csv')

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (58,59,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [203]:
# data_df.drop(columns=['ORDER_ID','ORDER_CREATED_DATE','ACCOUNT_NUMBER','SKU','PORT_IN_FLAG','FROM_CARRIER_NAME','BILLING_CITY','BILLING_STATE','BILLING_ZIPCODE','SHIPPING_CITY','SHIPPING_STATE','SHIPPING_ZIPCODE','EMAIL','IP_ADDRESS','BYOD_INDICATOR','HOLD_ORDER_STATUS','FINAL_RESULT','FINAL_RESULT_SOURCE','AW_REASON','SENT_TO_ACTIVE_WATCH_TIMESTAMP','ACTIVE_WATCH_TS','ORDER_SUBMIT_TS','ACTIVE_WATCH_DURATION_MINS','ACTIVE_WATCH_RESULT','EXPERIAN_RESPONSE_1','EXPERIAN_RESPONSE_2'], inplace=True)
# test_df.drop(columns=['ORDER_ID','ORDER_CREATED_DATE','ACCOUNT_NUMBER','SKU','PORT_IN_FLAG','FROM_CARRIER_NAME','BILLING_CITY','BILLING_STATE','BILLING_ZIPCODE','SHIPPING_CITY','SHIPPING_STATE','SHIPPING_ZIPCODE','EMAIL','IP_ADDRESS','BYOD_INDICATOR','HOLD_ORDER_STATUS','FINAL_RESULT','FINAL_RESULT_SOURCE','AW_REASON','SENT_TO_ACTIVE_WATCH_TIMESTAMP','ACTIVE_WATCH_TS','ORDER_SUBMIT_TS','ACTIVE_WATCH_DURATION_MINS','ACTIVE_WATCH_RESULT','EXPERIAN_RESPONSE_1','EXPERIAN_RESPONSE_2'], inplace=True)



data=data_df[['NUM_PORTIN','THIRD_PARTY_ID_SCORE','FIRST_PARTY_ID_SCORE','MAKE1','MONTHLYRECURRINGCHARGE','HOUR_OF_DAY','TOTAL_PRICE','PRICE1','ONETIMECHARGE','IDA_RESULT','MODEL1','INSTALLMENT_AMOUNT','IS_EXISTING_CUSTOMER','ORDER_HOLD_DURATION_MINS']]
test_data=test_df[['NUM_PORTIN','THIRD_PARTY_ID_SCORE','FIRST_PARTY_ID_SCORE','MAKE1','MONTHLYRECURRINGCHARGE','HOUR_OF_DAY','TOTAL_PRICE','PRICE1','ONETIMECHARGE','IDA_RESULT','MODEL1','INSTALLMENT_AMOUNT','IS_EXISTING_CUSTOMER','ORDER_HOLD_DURATION_MINS']]


In [204]:
data.shape

(133797, 14)

In [205]:
test_data.shape

(26321, 14)

In [206]:
data.head()

,NUM_PORTIN,THIRD_PARTY_ID_SCORE,FIRST_PARTY_ID_SCORE,MAKE1,MONTHLYRECURRINGCHARGE,HOUR_OF_DAY,TOTAL_PRICE,PRICE1,ONETIMECHARGE,IDA_RESULT,MODEL1,INSTALLMENT_AMOUNT,IS_EXISTING_CUSTOMER,ORDER_HOLD_DURATION_MINS
0,0,504,466,Apple,45.33,0,847.92,649.99,33.33,GREEN,iPhone XR,33.33,N,12.65
1,0,384,293,Motorola,4.99,0,129.96,119.99,4.99,GREEN,moto e5 play,4.99,Y,12.12
2,0,540,494,Samsung,98.66,0,1099.84,999.99,41.66,GREEN,Galaxy Note9,41.66,N,16.87
3,1,410,331,LG,44.49,0,834.36,779.99,32.49,GREEN,G8 ThinQTM,32.49,Y,24.13
4,0,348,409,Apple,0.00,0,0.00,0.00,0.00,GREEN,iPhone 6,0.00,N,10.75


In [207]:
def label_duration(row):
    duration = row['ORDER_HOLD_DURATION_MINS']
    if duration <= 60 :
        return 0
    elif duration >60 and duration <=120 :
        return 1
    else:
        return 2

In [208]:
data['label'] = data.apply(label_duration, axis=1)
data.drop(columns=['ORDER_HOLD_DURATION_MINS'], inplace=True)

test_data['label'] = test_data.apply(label_duration, axis=1)
test_data.drop(columns=['ORDER_HOLD_DURATION_MINS'], inplace=True)

In [209]:
data.head()

,NUM_PORTIN,THIRD_PARTY_ID_SCORE,FIRST_PARTY_ID_SCORE,MAKE1,MONTHLYRECURRINGCHARGE,HOUR_OF_DAY,TOTAL_PRICE,PRICE1,ONETIMECHARGE,IDA_RESULT,MODEL1,INSTALLMENT_AMOUNT,IS_EXISTING_CUSTOMER,label
0,0,504,466,Apple,45.33,0,847.92,649.99,33.33,GREEN,iPhone XR,33.33,N,0
1,0,384,293,Motorola,4.99,0,129.96,119.99,4.99,GREEN,moto e5 play,4.99,Y,0
2,0,540,494,Samsung,98.66,0,1099.84,999.99,41.66,GREEN,Galaxy Note9,41.66,N,0
3,1,410,331,LG,44.49,0,834.36,779.99,32.49,GREEN,G8 ThinQTM,32.49,Y,0
4,0,348,409,Apple,0.00,0,0.00,0.00,0.00,GREEN,iPhone 6,0.00,N,0


In [210]:
test_data.shape

(26321, 14)

In [211]:
test_data.head()

,NUM_PORTIN,THIRD_PARTY_ID_SCORE,FIRST_PARTY_ID_SCORE,MAKE1,MONTHLYRECURRINGCHARGE,HOUR_OF_DAY,TOTAL_PRICE,PRICE1,ONETIMECHARGE,IDA_RESULT,MODEL1,INSTALLMENT_AMOUNT,IS_EXISTING_CUSTOMER,label
0,1,451,393,Apple,0.00,16,1249.99,1249.99,1249.99,GREEN,iPhone 11 Pro Max,0.00,Y,2
1,0,306,441,Apple,97.08,16,1337.42,1249.99,52.08,GREEN,iPhone 11 Pro Max,52.08,Y,2
2,1,515,473,Apple,107.41,16,900.84,849.99,35.41,GREEN,iPhone 11,35.41,Y,2
3,0,547,302,Apple,12.00,16,1059.99,999.99,999.99,GREEN,iPhone 11 Pro,0.00,Y,2
4,0,425,396,Apple,61.66,16,399.84,399.99,16.66,GREEN,iPhone 7,16.66,Y,2


In [212]:
collections.Counter(data['label'])

Counter({0: 103870, 1: 15592, 2: 14335})

In [213]:
green_df = data[data['label'] == 0]
yellow_df = data[data['label'] == 1]
red_df = data[data['label'] == 2]

In [214]:
green_under_df = green_df.sample(15592)
red_over_df= red_df.sample(15592, replace=True)

In [215]:
balanced_data_df = pd.concat([green_under_df, yellow_df,red_over_df])

In [216]:
balanced_data_df.shape

(46776, 14)

In [217]:
X_train = balanced_data_df.drop(columns=['label'])
Y_train = balanced_data_df['label']

X_test = test_data.drop(columns=['label'])
Y_test = test_data['label']

In [218]:
X_train.shape

(46776, 13)

In [219]:
Y_train = np.array(Y_train)

In [220]:
Y_train.shape

(46776,)

In [221]:
X_test.shape

(26321, 13)

In [222]:
Y_test.shape

(26321,)

In [223]:
balanced_data_df.to_csv("data/final_balanced_data.csv",index=False)
test_data.to_csv("data/test_data.csv",index=False)

In [224]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'orderhold-Duration-ExternalCompute'

# project folder
project_folder = './ml_experiment/orderhold-Duration-ExternalCompute'
experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.65
Subscription ID,be564fde-136b-4709-b7b6-abfc0bdfc134
Workspace Name,xm-ml-workspace
Resource Group,xm-ml
Location,centralus
Project Directory,./ml_experiment/orderhold-Duration-ExternalCompute
Experiment Name,orderhold-Duration-ExternalCompute


In [225]:
train_dflow = dprep.read_csv('data/final_balanced_data.csv', infer_column_types=True)
train_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
NUM_PORTIN,FieldType.INTEGER,0.00,5.00,46776.00,0.00,46776.00,0.00,0.00,0.00,6,0.00,0.00,0.00,0.00,1.00,1.00,2.00,3.00,4.89,0.80,0.75,0.56,1.11,2.67,2.00,0.00
THIRD_PARTY_ID_SCORE,FieldType.INTEGER,228.00,999.00,46776.00,0.00,46776.00,0.00,0.00,0.00,682,249.41,331.94,331.74,371.42,426.81,503.95,663.85,827.46,999.00,450.13,113.04,12778.59,1.39,3.02,692.04,228.00
FIRST_PARTY_ID_SCORE,FieldType.INTEGER,90.00,999.00,46776.00,0.00,46776.00,0.00,0.00,0.00,629,149.16,270.18,270.14,313.54,371.52,442.99,575.67,675.49,999.00,386.63,105.94,11222.63,1.24,3.98,635.40,122.26
MAKE1,FieldType.STRING,Apple,Zagg,46776.00,0.00,46776.00,0.00,0.00,0.00,13,,,,,,,,,,,,,,,,
MONTHLYRECURRINGCHARGE,FieldType.DECIMAL,0.00,532.90,46776.00,0.00,46776.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,16.30,45.00,82.99,111.45,190.94,301.45,52.16,42.20,1780.97,1.32,4.62,182.42,0.00
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,46776.00,0.00,46776.00,0.00,0.00,0.00,24,0.00,2.00,1.98,13.00,17.00,20.00,23.00,23.00,23.00,14.99,6.83,46.72,-1.03,-0.06,23.00,2.98
TOTAL_PRICE,FieldType.DECIMAL,0.00,7838.20,46776.00,0.00,46776.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,14.02,657.06,1089.93,1533.87,2561.17,4337.92,658.99,601.37,361648.93,1.36,5.86,2664.38,0.00
PRICE1,FieldType.DECIMAL,0.00,1449.99,46776.00,8.00,46768.00,0.00,5.00,0.00,50,0.00,119.99,119.99,349.99,599.99,999.96,1249.99,1449.99,1449.99,634.72,378.85,143527.48,0.09,-0.89,1449.99,0.00
ONETIMECHARGE,FieldType.DECIMAL,0.00,3749.97,46776.00,0.00,46776.00,0.00,0.00,0.00,771,0.00,0.00,0.00,0.00,29.16,45.83,573.94,1157.25,1886.53,80.78,218.89,47913.40,4.86,30.46,106.84,0.00
IDA_RESULT,FieldType.STRING,,YELLOW,46776.00,0.00,46776.00,0.00,0.00,140.00,4,,,,,,,,,,,,,,,,


In [226]:
test_dflow = dprep.read_csv('data/test_data.csv', infer_column_types=True)
test_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
NUM_PORTIN,FieldType.INTEGER,0.00,5.00,26321.00,0.00,26321.00,0.00,0.00,0.00,6,0.00,0.00,0.00,0.00,1.00,1.00,2.00,3.00,4.74,0.80,0.72,0.51,1.07,2.91,2.00,0.00
THIRD_PARTY_ID_SCORE,FieldType.INTEGER,233.00,999.00,26321.00,0.00,26321.00,0.00,0.00,0.00,662,249.74,337.07,336.16,376.32,433.66,514.11,686.36,850.16,999.00,458.32,117.11,13715.46,1.34,2.60,718.38,233.00
FIRST_PARTY_ID_SCORE,FieldType.INTEGER,88.00,999.00,26321.00,0.00,26321.00,0.00,0.00,0.00,612,154.72,270.75,269.65,315.86,376.61,452.66,589.85,690.34,999.00,392.43,109.85,12067.45,1.14,3.21,653.65,128.83
MAKE1,FieldType.STRING,Apple,Zagg,26321.00,0.00,26321.00,0.00,0.00,0.00,13,,,,,,,,,,,,,,,,
MONTHLYRECURRINGCHARGE,FieldType.DECIMAL,0.00,485.40,26321.00,0.00,26321.00,0.00,0.00,0.00,>1000,0.00,12.00,12.00,29.28,54.67,86.65,116.30,194.25,307.70,59.10,40.60,1648.51,1.31,4.83,167.31,0.00
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,26321.00,0.00,26321.00,0.00,0.00,0.00,24,0.00,1.00,1.00,13.29,17.00,20.00,23.00,23.00,23.00,15.07,7.10,50.44,-1.08,-0.13,23.00,3.85
TOTAL_PRICE,FieldType.DECIMAL,0.00,7734.95,26321.00,0.00,26321.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,185.43,743.35,1092.08,1500.84,2683.91,4555.26,715.19,587.86,345573.60,1.60,8.59,2385.97,0.00
PRICE1,FieldType.DECIMAL,0.00,1449.99,26321.00,8.00,26313.00,0.00,0.00,0.00,47,0.00,144.16,143.99,449.99,699.99,999.99,1249.99,1439.20,1449.99,688.83,352.67,124375.34,-0.04,-0.62,1449.99,0.00
ONETIMECHARGE,FieldType.DECIMAL,0.00,3249.96,26321.00,0.00,26321.00,0.00,0.00,0.00,495,0.00,0.00,0.00,10.69,29.16,45.83,699.99,1163.64,1909.65,90.38,229.48,52663.12,4.33,23.63,92.69,0.00
IDA_RESULT,FieldType.STRING,,YELLOW,26321.00,0.00,26321.00,0.00,0.00,44.00,4,,,,,,,,,,,,,,,,


In [227]:
X_df = train_dflow.drop_columns(columns=['label'])
y_df = train_dflow.keep_columns(columns=['label'], validate_column_exists=True)
X_test_df = test_dflow.drop_columns(columns=['label'])
y_test_df = test_dflow.keep_columns(columns=['label'], validate_column_exists=True)

In [228]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

cts = ws.compute_targets
amlcompute_cluster_name = "XM-ML-Compute"  # Name your cluster

# Check if this compute target already exists in the workspace.
compute_target = cts[amlcompute_cluster_name]
found = False

# # Check if this compute target already exists in the workspace.

cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                vm_priority = 'lowpriority',
                                                                min_nodes = 0, max_nodes = 6)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

#     # Can poll for a minimum number of nodes and for a specific timeout.
#     # If no min_node_count is provided, it will use the scale settings for the cluster.
#     compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

#      # For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.


In [229]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# cd = CondaDependencies.create(conda_packages=['numpy','py-xgboost<=0.80'])
cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'psutil'], conda_packages=['numpy','py-xgboost<=0.80'])

conda_run_config.environment.python.conda_dependencies = cd

In [230]:
# Dump X and Y to CSVs.
X_df.to_pandas_dataframe().to_csv('data/duration/X_train.csv', index=False)
y_df.to_pandas_dataframe().to_csv('data/duration/Y_train.csv', index=False)

# Dump X and Y to CSVs.
X_test_df.to_pandas_dataframe().to_csv('data/duration/X_test.csv', index=False)
y_test_df.to_pandas_dataframe().to_csv('data/duration/Y_test.csv', index=False)

In [231]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data/duration', target_path='data/duration', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/duration/X_test.csv
Uploading ./data/duration/X_train.csv
Uploading ./data/duration/Y_test.csv
Uploading ./data/duration/Y_train.csv
Uploaded ./data/duration/Y_train.csv, 1 files out of an estimated total of 4
Uploaded ./data/duration/Y_test.csv, 2 files out of an estimated total of 4
Uploaded ./data/duration/X_test.csv, 3 files out of an estimated total of 4
Uploaded ./data/duration/X_train.csv, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_ea40c143f5ae47dda1b1cc7d4d7ad2af

In [232]:
X = Dataset.Tabular.from_delimited_files(path=ds.path('data/duration/X_train.csv'))
y = Dataset.Tabular.from_delimited_files(path=ds.path('data/duration/Y_train.csv'))

X_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/duration/X_test.csv'))
y_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/duration/Y_test.csv'))

In [233]:
automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             debug_log='automl_duration_errors.log',
                             path = project_folder, #For external compute
                             run_configuration=conda_run_config,
                             preprocess=True,
                             X = X, 
                             y = y,
                             X_valid=X_valid,
                             y_valid=y_valid,
                             iteration_timeout_minutes=10,
                             experiment_timeout_minutes=200,
                             max_concurrent_iterations=8,
                             iterations=50,
                             model_explainability=True
                             )

In [ ]:
ws = Workspace.from_config()
experiment_name = 'orderhold-Duration-ExternalCompute'
experiment = Experiment(ws, experiment_name)
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote compute: XM-ML-Compute
Parent Run ID: AutoML_0922b29d-f480-491a-8736-0c9eb97da5c5
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         3   MaxAbsScaler RandomForest                      0:08:28       0.4825    0.4825
         5   MaxAbsScaler ExtremeRandomTree

In [235]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
orderhold-Duration-ExternalCompute,AutoML_0922b29d-f480-491a-8736-0c9eb97da5c5,automl,Completed,Link to Azure Portal,Link to Documentation


In [236]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

A Jupyter Widget

In [237]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

ModuleNotFoundError: No module named 'pandas.core.internals.managers'; 'pandas.core.internals' is not a package

In [238]:
from azureml.train.automl.automlexplainer import retrieve_model_explanation

shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = \
    retrieve_model_explanation(best_run)

NameError: name 'best_run' is not defined

In [239]:
print(overall_summary)
print(overall_imp)
print(per_class_summary)
print(per_class_imp)

NameError: name 'overall_summary' is not defined